Fishing harvest parameters for the Salish Sea Atlantis Model

In [13]:
import os
import numpy as np
import requests
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from shapely.geometry import Point
import download_from_url

Collect fisheries landing data from 1990 to 2022 from DFO

In [14]:
def download_file(url, save_path):
    
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Save the file in chunks
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"File successfully downloaded and saved to {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [17]:
for year in range (1990, 2023):
    save_path = "/ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings"+str(year)+".xls"
    url = "https://www.dfo-mpo.gc.ca/stats/commercial/land-debarq/sea-maritimes/xls/s"+str(year)+"pq_e.xls"
    download_file(url, save_path)

File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1990.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1991.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1992.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1993.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1994.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1995.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1996.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seafisheries-landings1997.xls
File successfully downloaded and saved to /ocean/rlovindeer/Atlantis/Fisheries-data/Seaf